In [1]:
import pandas as pd
import re
import csv
from nltk import word_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import stem
from nltk import sent_tokenize


import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence
from sklearn.model_selection import train_test_split

from utils.text_cleaner import clean_text

2024-07-12 18:23:51 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2024-07-12 18:23:51 INFO: Downloading default packages for language: en (English) ...
2024-07-12 18:23:53 INFO: File exists: /Users/gbaldonado/stanza_resources/en/default.zip
2024-07-12 18:23:56 INFO: Finished downloading models and saved to /Users/gbaldonado/stanza_resources
2024-07-12 18:23:56 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-07-12 18:23:57 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2024-07-12 18:23:58 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-07-12 18:23:58 INFO: Using device: cpu
2024-07-12 18:23:58 INFO: Loading: tokenize
2024-07-12 18:23:58 INFO: Loading: mwt
2024-07-12 18:23:58 INFO: Loading: pos
2024-07-12 18:23:59 INFO: Loading: lemma
2024-07-12 18:23:59 INFO: Loading: constituency
2024-07-12 18:23:59 INFO: Loading: depparse
2024-07-12 18:23:59 INFO: Loading: sentiment
2024-07-12 18:24:00 INFO: Loading: 

ModuleNotFoundError: No module named 'utils'

##### Stanford Core NLP
* We need to download the [Stanford core nlp](https://stanfordnlp.github.io/CoreNLP/download.html) package before we can start with the dataset creation process.  
``
cd /Users/krishns18/SFSU/Modeling_ALMA/corenlp/stanford-corenlp-4.0.0
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
``   
* The server will start and the command prompt will display like below   
``
(base) Shaileshs-MBP:stanford-corenlp-4.0.0 krishns18$ java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
[main] INFO CoreNLP - --- StanfordCoreNLPServer#main() called ---
[main] INFO CoreNLP - Warning: cannot find edu/stanford/nlp/models/srparser/englishSR.ser.gz
[main] INFO CoreNLP - Setting default constituency parser to PCFG parser: edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz
[main] INFO CoreNLP - To use shift reduce parser download English models jar from:
[main] INFO CoreNLP - https://stanfordnlp.github.io/CoreNLP/download.html
[main] INFO CoreNLP -     Threads: 16
[main] INFO CoreNLP - Starting server...
[main] INFO CoreNLP - StanfordCoreNLPServer listening at /0:0:0:0:0:0:0:0:9000
``

In [34]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

## 1.) Load dataset

In [ ]:
# Load the Excel workbook
published_file_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR.xlsx'

xls = pd.ExcelFile(published_file_path)
# Loop through each sheet and print the header
for sheet_name in xls.sheet_names:
    df = pd.read_excel(published_file_path, sheet_name=sheet_name)
    print(f"'{sheet_name}':")
    print(list(df.columns))



'S18 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Filial Piety FINAL', 'Familial FINAL', 'Community Consciousness FINAL ', 'Social FINAL', 'Spiritual FINAL']
'F18 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Filial Piety FINAL', 'Familial FINAL', 'Community Consciousness FINAL ', 'Social FINAL', 'Spiritual FINAL']
'S19 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Filial Piety FINAL', 'Familial FINAL', 'Community Consciousness FINAL ', 'Social FINAL', 'Spiritual FINAL']
'F19 SI':
['ID CODE', 'Essay: Why am I here?', 'Attainment', 'First Gen FINAL', 'Aspirational FINAL', 'Navigational FINAL', 'Resistance FINAL', 'Perserverance FINAL', 'Fili

In [ ]:
desired_column_names = ['Alma ID', 
                        'Essay: Why I am here?', 
                        'Attainment', 
                        'First Gen', 
                        'Aspirational', 
                        'Navigational', 
                        'Resistance', 
                        'Perseverance', 
                        'Filial Piety', 
                        'Familial', 
                        'Community Consciousness',
                        'Social', 
                        'Spiritual']

In [ ]:
def update_headers(file_path, output_path, desired_header):
    """
    Updates the headers of all sheets in an Excel workbook to the specified header.

    :param file_path: Path to the input Excel file.
    :param output_path: Path to the output Excel file.
    :param desired_header: List of desired headers.
    """
    # Load the Excel workbook
    xls = pd.ExcelFile(file_path)

    # Create a dictionary to store updated DataFrames
    updated_sheets = {}

    # Loop through each sheet and update the header
    for sheet_name in xls.sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
        print(f"Current headers for sheet '{sheet_name}':")
        print(list(df.iloc[0]))
        
        # Replace the existing header with the desired header
        df.columns = desired_header
        df = df[1:]  # Remove the original header row
        updated_sheets[sheet_name] = df

    # Save the updated sheets back to an Excel workbook
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name, df in updated_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)


# published_file_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR.xlsx'
# output_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR_header_fixed.xlsx'

unpublished_file_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_unpublished_quotes_IRR.xlsx'
output_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_unpublished_quotes_IRR_header_fixed.xlsx'

update_headers(unpublished_file_path, output_path, desired_column_names)

Current headers for sheet 'Pilot':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Current headers for sheet 'Spring 2018 SI 333':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Current headers for sheet 'Spring 2018 - SI':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Current headers for sheet 'Fall 2019 - PHYS102':
['Alma ID', 'Essay: Why I am here?', 'Attainment', 'First Gen', 'Aspirational', 'Navigational', 'Resistance', 'Perseverance', 'Filial Piety', 'Familial', 'Community Consciousness', 'Social', 'Spiritual']
Curren

In [3]:
# Read the essays and create a dataframe
df_1 = pd.concat(pd.read_excel('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_published_quotes_IRR_header_fixed.xlsx', sheet_name=None),ignore_index=True,sort=False)
print(df_1.shape)

(500, 13)


In [4]:
# Read the essays and create a dataframe
df_2 = pd.concat(pd.read_excel('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1_human_annotated/all_capitals_unpublished_quotes_IRR_header_fixed.xlsx', sheet_name=None),ignore_index=True,sort=False)
print(df_2.shape)

(89, 13)


In [5]:
df = pd.concat([df_1, df_2])

In [6]:
df

,Alma ID,Essay: Why I am here?,Attainment,First Gen,Aspirational,Navigational,Resistance,Perseverance,Filial Piety,Familial,Community Consciousness,Social,Spiritual
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...,0,to better one self and be able to succeed late...,0,0,0,0,0,0,0,I always wanted to be able to help people
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0,0,I am also here because the formula to being su...,0,0,0,She made a lot of sacrifices for me to be here...,I'm here because my parents came here during t...,0,0,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,0,0,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...,I know that as a child I never thought of edu...,0,I am here because I want to better myself & my...,0,I Hope to reach a position in which I can insp...,0,0
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...,0,0,School has always helped me escape the hard th...,"As the oldest of 3 siblings, I want to be a ro...",0,I want to help my community while I'm here,0,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,0,0,- I am here to help myself grow. /%/ - Unders...,- Learn and apply my skills to progress toward...,0,0,0,0,0,Meet people who have (similar) aspirations/goa...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0,0,0,0,0,0,0,My family has valued hard work. Growing up I w...,0,0,0
85,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0,0,0,0,0,0,This is something I plan on doing with my kids.,my dad told me that it’s going to be hard to g...,0,0,0
86,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0,0,0,0,0,0,0,0,The values of my community have made a deep an...,0,0
87,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0,0,0,0,0,0,0,Well one thing my mom has taught me is to not ...,0,0,0


In [8]:
df.to_csv("batch_1_dataset_all_capitals_quotes_published_unpublished_IRR.csv", index=False)

**Creating a separate dataset for Attainment**

In [39]:
df

,Alma ID,Essay: Why I am here?,Attainment,First Gen,Aspirational,Navigational,Resistance,Perseverance,Filial Piety,Familial,Community Consciousness,Social,Spiritual
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...,0,to better one self and be able to succeed late...,0,0,0,0,0,0,0,I always wanted to be able to help people
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0,0,I am also here because the formula to being su...,0,0,0,She made a lot of sacrifices for me to be here...,I'm here because my parents came here during t...,0,0,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,0,0,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...,I know that as a child I never thought of edu...,0,I am here because I want to better myself & my...,0,I Hope to reach a position in which I can insp...,0,0
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...,0,0,School has always helped me escape the hard th...,"As the oldest of 3 siblings, I want to be a ro...",0,I want to help my community while I'm here,0,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,0,0,- I am here to help myself grow. /%/ - Unders...,- Learn and apply my skills to progress toward...,0,0,0,0,0,Meet people who have (similar) aspirations/goa...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0,0,0,0,0,0,0,My family has valued hard work. Growing up I w...,0,0,0
85,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0,0,0,0,0,0,This is something I plan on doing with my kids.,my dad told me that it’s going to be hard to g...,0,0,0
86,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0,0,0,0,0,0,0,0,The values of my community have made a deep an...,0,0
87,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0,0,0,0,0,0,0,Well one thing my mom has taught me is to not ...,0,0,0


## 2. Create theme datasets

In [51]:
def create_theme_datasets(df, themes, sentence_level=True):
    """
    Creates separate datasets for each theme and returns them as a dictionary.

    :param df: The original DataFrame.
    :param themes: List of theme columns to create separate datasets for.
    :return: A dictionary with theme names as keys and DataFrames as values.
    """
    base_columns = ['Alma ID', 'Essay: Why I am here?']
    theme_datasets = {}

    for theme in themes:
        # Select the columns for the theme dataset
        theme_df = df[base_columns + [theme]]

        if sentence_level:
        # Filter out rows where the theme value is 0
            theme_df = theme_df[theme_df[theme] != 0]
        
        # Add the DataFrame to the dictionary
        theme_datasets[theme] = theme_df

    return theme_datasets

theme_names = desired_column_names[2:]

# Create the theme datasets
sentence_level_theme_datasets = create_theme_datasets(df, theme_names, True)
essay_level_theme_datasets = create_theme_datasets(df, theme_names, False)

In [52]:
sentence_level_theme_datasets["Attainment"]

,Alma ID,Essay: Why I am here?,Attainment
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
6,S18.W2.111.01.07,I am here to better myself and hopefully give ...,I am in this class because I want to pass phys...
8,S18.W2.111.01.09,I am here in this class because I want a littl...,I am majoring in bio to get to my career goal ...
11,S18.W2.111.01.12,I am here at SFSU for a better future. A futur...,I aspire to become a doctor /%/ To achieve goa...
16,S18.W2.111.01.17,I'm here because I want to improve my understa...,"In the end, the goal is to achieve an A for my..."
...,...,...,...
68,F19.PHYS102.02.08,Why am I here? If you're asking me why am I on...,I would tell you that I am chasing a degree. I...
72,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...
73,F19.PHYS242.05.02,I am here because I want to make my mom proud....,I want to graduate
75,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...


In [54]:
essay_level_theme_datasets["Attainment"]

,Alma ID,Essay: Why I am here?,Attainment
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,0
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,0
...,...,...,...
84,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
85,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
86,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
87,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [8]:
attainment_df = df.copy()

In [9]:
# dropping the first gen column from attainment dataset.
attainment_df = attainment_df.drop(columns=['First Gen - FINAL'])
attainment_df.head()

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
1,pilot_study_02,I want to stay in my community & help educate ...,0
2,pilot_study_03,I am here to provide myself with opportunities...,0
3,pilot_study_04,I am here to figure out what I want to do with...,0
4,pilot_study_05,I am here at San Francisco State because I wan...,0


In [10]:
# drop essays which do not contain attainment
attainment_df = attainment_df[~attainment_df['Attainment - FINAL'].isin([0, '-', 'Unable to code'])]

In [11]:
attainment_df = attainment_df.dropna() # drop null

In [12]:
attainment_df.shape

(185, 3)

In [13]:
attainment_df

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
8,pilot_study_09,I’m here to learn and to be on the path toward...,I’m here to learn and to be on the path toward...
9,pilot_study_10,I am in SFSU because I want to go to medical s...,I am in SFSU because I want to go to medical s...
11,pilot_study_12,I am here in SFSU because I am the first gener...,My goal is to graduate with a degree B.A./B.S.
13,pilot_study_14,I am here because of my goal in trying to get ...,I am here because of my goal in trying to get ...
...,...,...,...
551,F19.PHYS232.07.23,I'm here because of prerequisite for my comput...,And my dream job is to be a software engineer.
559,F19.PHYS232.09.08,The reason I am here is to do experiments and ...,I am an aspiring Engineer and I love doing ex...
561,F19.PHYS232.09.10,I am in college to further my education. There...,I am studying to become an electrical engineer.
562,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...


In [14]:
# total number of essays
print("Total number of essays containing attainment are: {}".format(attainment_df.shape[0]))

Total number of essays containing attainment are: 185


**Creating dataset for first generation**

In [15]:
first_generation_df = df.copy()

In [16]:
# dropping the attainment column from the dataset.
first_generation_df = first_generation_df.drop(columns=['Attainment - FINAL'])

In [17]:
first_generation_df.shape

(570, 3)

In [18]:
# drop essays which do not contain first generation
first_generation_df = first_generation_df[~first_generation_df['First Gen - FINAL'].isin([0, '-', 'Unable to code'])]

In [19]:
first_generation_df.shape

(20, 3)

In [20]:
first_generation_df = first_generation_df.dropna() # drop null

In [21]:
first_generation_df.shape

(19, 3)

In [22]:
# total number of essays
print("Total number of essays containing first generation thematic code are: {}".format(first_generation_df.shape[0]))

Total number of essays containing first generation thematic code are: 19


**Data Preparation**  
1. Get noun and adjective

In [61]:
def get_noun_adj(phrase):
    noun_adj = []
    for tag in pos_tag(phrase.split()):
        if tag[1] in ['NN','NNP','NNS', 'NNPS','JJ', 'JJR', 'JJS']:
            noun_adj.append(tag[0])
    return  noun_adj

2. Get NER from sentence
    - Need to have the corenlp server up and running

In [168]:
# Need to have corenlp server running
def get_ner(text):
    doc = nlp(text)
    ner_list = [ent.text for sent in doc.sentences for ent in sent.ents]
    return ner_list

In [172]:
def get_ner(text):
    ner_list = []
    # Annotate the text using stanza
    doc = nlp(text)

    for sentence in doc.sentences:
        for entity in sentence.ents:
            if entity.type == 'PERSON':
                ner_list.append(entity.text)

    return ner_list

# Example usage
text = "Barack Obama was the 44th President of the United States."
print(get_ner(text))

['Barack Obama']


In [239]:
## to replace get_ner??
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

def jaccard_similarity(str1, str2):
    vect = CountVectorizer().fit_transform([str1, str2])
    vectors = vect.toarray()
    return jaccard_score(vectors[0], vectors[1], average='weighted')


In [233]:
sample_sentence = "I don't know if this question -is pertaining -to why am I -here at San Francisco State University or why am I here in this class. I am going to answer both. I am here at San Francisco State University because I wanted to be able to be far from home, but not too far. A one hour airplane trip back home, 7 hour drive. I wanted to experience a different environment and a different culture because not everything done in SoCal is done in NorCal. I am in college specifically to pursue my one and only career which is to be a veterinarian. Getting declined from UC Davis because they have a great vet program lead me over here where I can get my degree then try UC Davis later. I am here to get my degree, network, and to make great friendships. Why am I here in this Sci 111 course? I am in this course because  I am new to physics I never took physics in high school because I was told it can mess up my GPA. Hearing that I already knew it would be a challenging course. I am in the course to get more help with understanding the concept and formulas of physics because the first class was very confusing and my professor teaches very fast. What I hope to get out of this Sci class is a clear understanding of how to solve and physics problem and be confident that I know how to do it."

get_ner(sample_sentence)
get_noun_adj(sample_sentence)

['question',
 '-to',
 'San',
 'Francisco',
 'State',
 'University',
 'class.',
 'both.',
 'San',
 'Francisco',
 'State',
 'University',
 'able',
 'home,',
 'A',
 'hour',
 'airplane',
 'trip',
 'hour',
 'drive.',
 'different',
 'environment',
 'different',
 'culture',
 'everything',
 'SoCal',
 'NorCal.',
 'college',
 'career',
 'veterinarian.',
 'Getting',
 'UC',
 'Davis',
 'great',
 'vet',
 'program',
 'lead',
 'degree',
 'UC',
 'Davis',
 'later.',
 'degree,',
 'network,',
 'great',
 'friendships.',
 'Sci',
 'course?',
 'course',
 'new',
 'physics',
 'physics',
 'high',
 'school',
 'GPA.',
 'Hearing',
 'challenging',
 'course.',
 'course',
 'more',
 'help',
 'concept',
 'formulas',
 'physics',
 'first',
 'class',
 'confusing',
 'professor',
 'teaches',
 'Sci',
 'class',
 'clear',
 'understanding',
 'problem',
 'confident']

In [140]:
get_ner("I am in college specifically to pursue my one and only career which is to be a veterinarian.")
get_noun_adj("I am in college specifically to pursue my one and only career which is to be a veterinarian.")

['college', 'career', 'veterinarian.']

3. Get sentence label
    1. Look for exact match
    2. Check for NER in phrase (title/profession/scientific fields) and check if those exist in the sentence
    3. Get the nouns and adjectives and check if those exist in the sentence

In [240]:
def get_sentence_label(essay_sentence, phrase): 
    # 1. look for the exact match of the phrase in the sentence
    match = re.search('.*%s.*' %phrase,essay_sentence, re.IGNORECASE)
    if match:
        return 1
    else:        
        # 2. check for NER in phrase (title /profession/ scientific fields) check if those exist in sentence
        ner_list = get_ner(phrase)
        ner_found_flag = None
        if len(ner_list) == 0:
            ner_found_flag = 0
        else: 
            for ner in ner_list:
                if not (re.search('.*%s.*' %ner, essay_sentence)):
                    ner_found_flag = 0
                    break
                else:
                    ner_found_flag = 1
        if ner_found_flag:
            return 1
        else:
            # 3. Else get the nouns and adjectives and check if those present in the sentence
            noun_adj_phrase = get_noun_adj(phrase)
            all_pos_found = None
            for pos in noun_adj_phrase:
                if pos not in essay_sentence:
                    all_pos_found = 0
                    break
                else:
                    all_pos_found = 1
            if all_pos_found == 1:
                return 1
            else:
                return 0

In [301]:
text = "I‚Äôm here because I want to become a dentist. How about you? Let's do this!"
cleaned_text = clean_text(text)
cleaned_text

"Im here because I want to become a dentist. How about you? Let's do this!"

In [303]:
## updated with jaccard sim and removed get_ner

def get_sentence_label(essay_sentence, phrase): 
    # 1. look for the exact match of the phrase in the sentence
    match = re.search('.*%s.*' %phrase,essay_sentence, re.IGNORECASE)
    if match:
        return 1
    else:        
        similarity = jaccard_similarity(essay_sentence, phrase)
        # print(similarity)
        if similarity > 0.5:  # Adjust the threshold as needed
            return 1
        else:
            # 3. Else get the nouns and adjectives and check if those present in the sentence
            noun_adj_phrase = get_noun_adj(phrase)
            all_pos_found = None
            for pos in noun_adj_phrase:
                if pos not in essay_sentence:
                    all_pos_found = 0
                    break
                else:
                    all_pos_found = 1
            if all_pos_found == 1:
                return 1
            else:
                return 0

# def get_sentence_label_with_jaccard(essay_sentence, phrases):
#     # 1. Look for the exact match of the phrase in the sentence
#     for phrase in phrases:
#         # Create regex pattern for exact match with word boundaries
#         pattern = r'\b%s\b' % re.escape(phrase)
#         match = re.search(pattern, essay_sentence, re.IGNORECASE)
#         if match:
#             print("Exact match found:", phrase)
#             return 1
    
#     # 2. Get the nouns and adjectives and check if those are present in the sentence
#     noun_adj_phrase = get_noun_adj(phrases)
#     all_pos_found = all(pos.lower() in essay_sentence.lower() for pos in noun_adj_phrase)
#     if all_pos_found:
#         "pos"
#         return 1
    
#     # 3. Use Jaccard similarity to find similarity between sentence and phrase
#     for phrase in phrases:
        
    
#     return 0

In [201]:
get_sentence_label("a one hour airplane trip back home, 7 hour drive",'I am in college specifically to pursue my one and only career which is to be a veterinarian')

0.015037593984962405


0

In [191]:
get_sentence_label_jaccard("i√≠m not shy to say it how it is and it will drive me to success",'The only way that I√≠m able to create a better future for my community is by getting a PhD')

0.015384615384615385


0

In [190]:
get_sentence_label_jaccard("i am hoping for an a, and i honestly think it is do-able because of all the extra time i am going to be putting in to make sure i understand all the concepts","I am hoping for an A" )

1

In [196]:
from utils.custom_tokenizer import custom_sent_tokenizer

In [197]:

text = "Dr. Smith is a veterinarian. P.A. Jones assists him."
sentences = custom_sent_tokenizer(text)
print(sentences)

['Dr. Smith is a veterinarian.', 'P.A. Jones assists him.']


In [211]:
sent = "‚Ä†i am at san francisco state university to get a bachelors degree in civil engineering "
sentences = clean_text(sent)
sentences = custom_sent_tokenizer(sentences)

print(sentences)



['i am at san francisco state university to get a bachelors degree in civil engineering ']


In [209]:
clean_text(sent)

'Ph.D. I want to get it. I need to get my p.a.. first though. Mr. baldonado is done. N.A.S.A. wants use to the l.a.s.e.r. technology? No way! I would love D.r.? '

## Looking at example 1, index 0

In [64]:
# entire essay
attainment_df["Essay : Why am I here"][0]

NameError: name 'attainment_df' is not defined

In [31]:
#sentencep positive for attainment
attainment_df["Attainment - FINAL"][0]

'I am here to get my BS in physiology so I can get into medical school.'

## 3. Creating sentence level phrases

In [218]:
sent = "i feel like i wasn‚Äôt created to be a cardiothoracic surgeon."
sent = 
clean_text(sent)

'i feel like i wasnt created to be a cardiothoracic surgeon.'

In [310]:
annotation = "- I am here to help myself grow. /%/  - Understand my purpose. /%/  Improve /%/ - Learn and apply my skills to progress towards my career goal."

annotation_list = re.split(r'/%/', annotation)
annotation_list = [clean_text(text) for text in annotation_list]
annotation_list

['I am here to help myself grow.',
 'Understand my purpose.',
 'Improve',
 'Learn and apply my skills to progress towards my career goal.']

In [313]:
def process_sentences_for_theme(theme_df, theme_column):
    theme_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
    
    for index, val in theme_df.iterrows():
        text = val['Essay: Why I am here?'].rstrip('\n\r')
        
        # Split the paragraphs into sentences using sent_tokenize from nltk
        temp_text = custom_sent_tokenizer(text)
        
        # Annotation
        annotation = val[theme_column]
         
        # Annotation list
        annotation_list = re.split(r'/%/', annotation)
        annotation_strip = [clean_text(annotation) for annotation in annotation_list]
        essay_strip = [x for x in temp_text if x != '']
        
        # Remove extra spaces before the sentences
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # Annotate the essay on sentence level
        for sentence in essay_strip:
            sentence = clean_text(sentence)
            if not re.search(r'\w', sentence):
                continue
            if len(sentence.split(' ')) < 3:
                continue
            
            label = 0
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence, annotation_phrase)
                    if label == 1:
                        break
            
            # Create a new DataFrame with the row you want to add
            new_row = pd.DataFrame([{'sentence': sentence.lower(), 'label': label, 'phrase': annotation_strip}])
            
            # Concatenate the new row to the existing DataFrame
            theme_sentence_df = pd.concat([theme_sentence_df, new_row], ignore_index=True)
    
    return theme_sentence_df

In [96]:
sentence_level_theme_names = ["Attainment", "Aspirational", "Familial", "Filial Piety", "First Gen", "Community Consciousness", "Spiritual", "Social"]

In [98]:
sentence_level_theme_names[1:]

['Aspirational',
 'Familial',
 'Filial Piety',
 'First Gen',
 'Community Consciousness',
 'Spiritual',
 'Social']

In [213]:
theme_datasets["Attainment"]

,Alma ID,Essay: Why I am here?,Attainment
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
6,S18.W2.111.01.07,I am here to better myself and hopefully give ...,I am in this class because I want to pass phys...
8,S18.W2.111.01.09,I am here in this class because I want a littl...,I am majoring in bio to get to my career goal ...
11,S18.W2.111.01.12,I am here at SFSU for a better future. A futur...,I aspire to become a doctor /%/ To achieve goa...
16,S18.W2.111.01.17,I'm here because I want to improve my understa...,"In the end, the goal is to achieve an A for my..."
...,...,...,...
68,F19.PHYS102.02.08,Why am I here? If you're asking me why am I on...,I would tell you that I am chasing a degree. I...
72,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...
73,F19.PHYS242.05.02,I am here because I want to make my mom proud....,I want to graduate
75,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...


In [220]:
attainment_processed_sentence_df_2 = process_sentences_for_theme(theme_datasets["Attainment"], "Attainment")

0.0
0.0
0.01566951566951567
0.03209876543209876
0.03550295857988166
0.043478260869565216
0.1
0.09286412512218964
0.05407407407407407
0.057692307692307696
0.03819058212829069
0.03921568627450981
0.0
0.0
0.0
0.0
0.0
0.0
0.0234375
0.1323529411764706
0.05263157894736842
0.07692307692307693
0.019047619047619046
0.021683673469387755
0.0
0.022675736961451247
0.04301075268817204
0.0
0.017361111111111112
0.9285714285714286
0.020833333333333332
0.0
0.05555555555555555
0.02040816326530612
0.0510204081632653
0.22307692307692306
0.027700831024930747
0.045454545454545456
0.0625
0.023391812865497075
0.0
0.02287581699346405
0.0
0.0
0.21484375
0.020833333333333332
0.0
0.06432748538011696
0.026915113871635612
0.029411764705882353
0.03703703703703704
0.0154320987654321
0.05119047619047619
0.015037593984962405
0.038461538461538464
0.016693163751987282
0.11038961038961038
0.0380952380952381
0.0343980343980344
0.05714285714285714
0.036053130929791274
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.041666666666666664
0.0
0.0


In [224]:
attainment_processed_sentence_df["label"].value_counts()

label
0    1437
1     304
Name: count, dtype: int64

In [223]:
attainment_processed_sentence_df_2["label"].value_counts()

label
0    1407
1     261
Name: count, dtype: int64

In [221]:
attainment_processed_sentence_df_2
attainment_processed_sentence_df_2.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/atn_sentence_level_batch_1_jaccard.csv", index=False)

In [314]:
processed_theme_datasets = {}

for i in sentence_level_theme_names: #skipping Attainment because we already did it
    print("Working on " + i)
    processed_name = f"{i}_processed_sentence_df"
    processed_theme_datasets[i] = process_sentences_for_theme(sentence_level_theme_datasets[i], i)

Working on Attainment
Working on Aspirational
Working on Familial
Working on Filial Piety
Working on First Gen
Working on Community Consciousness
Working on Spiritual
Working on Social


In [299]:
from utils.text_cleaner import clean_text



In [305]:
for k,v in sentence_level_theme_datasets.items():
    print(k, len(v))

Attainment 220
First Gen 21
Aspirational 393
Navigational 401
Resistance 21
Perseverance 126
Filial Piety 82
Familial 80
Community Consciousness 29
Social 101
Spiritual 71


In [230]:
aspirational_processed_sentence_df = processed_theme_datasets["Aspirational"]
aspirational_processed_sentence_df.to_csv("")

,sentence,label,phrase
0,why am i here?,0,[to better one self and be able to succeed lat...
1,well why does anyone pursue a higher education?,0,[to better one self and be able to succeed lat...
2,to better one self and be able to succeed late...,1,[to better one self and be able to succeed lat...
3,ever since i was little i wanted to be a docto...,0,[to better one self and be able to succeed lat...
4,i always wanted to be able to help people and ...,1,[to better one self and be able to succeed lat...
...,...,...,...
3031,"i want to better myself, improve myself in eve...",1,[I want to be able to apply my knowledge to my...
3032,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3033,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3034,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


In [315]:
for k,v in processed_theme_datasets.items():
    print(k, len(v))
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/using_jaccard_method/{k}_sentence_level_batch_1_jaccard.csv", index=False)

Attainment 1668
Aspirational 3036
Familial 640
Filial Piety 659
First Gen 141
Community Consciousness 186
Spiritual 592
Social 847


## 3. Merging similar themes 

1. Attainment and aspirational
2. Familial, filial piety, and first-generation
3. Perserverance and resistance (essay level)
4. Community consciousness, spiritual, and social
5. Navigational (essay level)


In [316]:
theme_names

['Attainment',
 'First Gen',
 'Aspirational',
 'Navigational',
 'Resistance',
 'Perseverance',
 'Filial Piety',
 'Familial',
 'Community Consciousness',
 'Social',
 'Spiritual']

In [323]:
[processed_theme_datasets[i] for i in merged_themes["Aspirational"]]

,sentence,label,phrase
0,why am i here?,0,[to better one self and be able to succeed lat...
1,well why does anyone pursue a higher education?,0,[to better one self and be able to succeed lat...
2,to better one self and be able to succeed late...,1,[to better one self and be able to succeed lat...
3,ever since i was little i wanted to be a docto...,0,[to better one self and be able to succeed lat...
4,i always wanted to be able to help people and ...,0,[to better one self and be able to succeed lat...
...,...,...,...
3031,"i want to better myself, improve myself in eve...",0,[I want to be able to apply my knowledge to my...
3032,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3033,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3034,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


In [324]:


merged_themes = {"Aspirational": ["Attainment", "Aspirational"],
                 "Familial": ["Familial", "Filial Piety", "First Gen"],
                 "Social": ["Community Consciousness", "Spiritual", "Social"]}
merged_aspirational_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Aspirational"]])
merged_familial_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Familial"]])
merged_social_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Social"]])



In [328]:
len(attainment_processed_sentence_df[attainment_processed_sentence_df["label"] == 1])

304

In [333]:
merged_sentence_level_datasets = {"Aspirational": merged_aspirational_df,
                                  "Familial": merged_familial_df,
                                  "Social": merged_social_df
                                  }

for k, v in merged_sentence_level_datasets.items():
    print(k, v["label"].value_counts(), f"percentage of themed essays: {round(len(v[v['label'] == 1]) / len(v) * 100, 2)}%")


Aspirational label
0    4116
1     588
Name: count, dtype: int64 percentage of themed essays: 12.5%
Familial label
0    1283
1     157
Name: count, dtype: int64 percentage of themed essays: 10.9%
Social label
0    1440
1     185
Name: count, dtype: int64 percentage of themed essays: 11.38%


In [335]:
for k,v in merged_sentence_level_datasets.items():
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_{k}_sentence_level_batch_1_jaccard.csv", index=False)

In [118]:
processed_theme_datasets.keys()

dict_keys(['Aspirational_processed_sentence_df', 'Familial_processed_sentence_df', 'Filial Piety_processed_sentence_df', 'First Gen_processed_sentence_df', 'Community Consciousness_processed_sentence_df', 'Spiritual_processed_sentence_df', 'Social_processed_sentence_df'])

In [119]:
atn_asp_df = pd.concat([attainment_processed_sentence_df, aspirational_processed_sentence_df])
atn_asp_df

,sentence,label,phrase
0,why am i here,0,[Ever since I was little I wanted to be a doct...
1,well why does anyone pursue a higher education,0,[Ever since I was little I wanted to be a doct...
2,to better one self and be able to succeed late...,0,[Ever since I was little I wanted to be a doct...
3,ever since i was little i wanted to be a docto...,1,[Ever since I was little I wanted to be a doct...
4,i always wanted to be able to help people and ...,0,[Ever since I was little I wanted to be a doct...
...,...,...,...
3151,"i want to better myself, improve myself in eve...",1,[I want to be able to apply my knowledge to my...
3152,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3153,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3154,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti


In [127]:
atn_asp_df.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/atn_asp_sentence_level_batch_1.csv", index=False)


In [56]:
attainment_sentence_df = sentence_level_theme_datasets["Attainment"]
attainment_sentence_df

,Alma ID,Essay: Why I am here?,Attainment
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
6,S18.W2.111.01.07,I am here to better myself and hopefully give ...,I am in this class because I want to pass phys...
8,S18.W2.111.01.09,I am here in this class because I want a littl...,I am majoring in bio to get to my career goal ...
11,S18.W2.111.01.12,I am here at SFSU for a better future. A futur...,I aspire to become a doctor /%/ To achieve goa...
16,S18.W2.111.01.17,I'm here because I want to improve my understa...,"In the end, the goal is to achieve an A for my..."
...,...,...,...
68,F19.PHYS102.02.08,Why am I here? If you're asking me why am I on...,I would tell you that I am chasing a degree. I...
72,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...
73,F19.PHYS242.05.02,I am here because I want to make my mom proud....,I want to graduate
75,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...


In [78]:
attainment_processed_sentence_df = process_sentences_for_theme(attainment_sentence_df, "Attainment")

KeyboardInterrupt: 

In [94]:
attainment_processed_sentence_df[14:]

,sentence,label,phrase
14,i knew this step was going to be an important ...,0,[I am majoring in bio to get to my career goal...
15,i am here at sfsu for a better future,0,"[I aspire to become a doctor , To achieve goal..."
16,a future where i don't have to struggle at the...,0,"[I aspire to become a doctor , To achieve goal..."
17,to achieve goals such as open up a homeless sh...,1,"[I aspire to become a doctor , To achieve goal..."
18,i aspire to become a doctor because i know tha...,1,"[I aspire to become a doctor , To achieve goal..."
...,...,...,...
1736,†by the time i got here at sfsu i decided to j...,0,[I am at San Francisco State University to get...
1737,†right now i am slightly considering switching...,0,[I am at San Francisco State University to get...
1738,one of the main reasons why i came to californ...,1,[One of the main reasons why I came to Califor...
1739,i move to san francisco because i got accepted...,0,[One of the main reasons why I came to Califor...


In [93]:
attainment_processed_sentence_df["sentence"][18]

'i aspire to become a doctor because i know that not only can i change my life, families, but also the people around me'

**Generate csv file for Attainment**

In [131]:
attainment_df = theme_datasets["Attainment"]

In [132]:
attainment_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/using_original_method/attainment_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in attainment_df.iterrows():
        text = val['Essay: Why I am here?'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['Attainment']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            # Create a new DataFrame with the row you want to add
            new_row = pd.DataFrame([{'sentence': sentence.lower(), 'label': label, 'phrase': annotation_strip}])

            # Concatenate the new row to the existing DataFrame
            attainment_sentence_df = pd.concat([attainment_sentence_df, new_row], ignore_index=True)
            # attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)

In [59]:
attainment_df

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
8,pilot_study_09,I’m here to learn and to be on the path toward...,I’m here to learn and to be on the path toward...
9,pilot_study_10,I am in SFSU because I want to go to medical s...,I am in SFSU because I want to go to medical s...
11,pilot_study_12,I am here in SFSU because I am the first gener...,My goal is to graduate with a degree B.A./B.S.
13,pilot_study_14,I am here because of my goal in trying to get ...,I am here because of my goal in trying to get ...
...,...,...,...
551,F19.PHYS232.07.23,I'm here because of prerequisite for my comput...,And my dream job is to be a software engineer.
559,F19.PHYS232.09.08,The reason I am here is to do experiments and ...,I am an aspiring Engineer and I love doing ex...
561,F19.PHYS232.09.10,I am in college to further my education. There...,I am studying to become an electrical engineer.
562,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...


In [58]:
attainment_sentence_df

,sentence,label,phrase


In [27]:
pos_labels = len([n for n in attainment_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(attainment_sentence_df['label']), (pos_labels/len(attainment_sentence_df['label']))*100))


Positive labels present in the dataset : 253  out of 1413 or 17.905166312809627%


In [28]:
train, test = train_test_split(attainment_sentence_df, test_size=0.1, random_state=18, stratify=attainment_sentence_df['label'])

In [29]:
train.to_csv("data/csv/attainment_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [30]:
test.to_csv("data/csv/attainment_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [31]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 17.94% positive class
positive labels : 228 out of total 1271 datapoints


In [32]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 17.61% positive class
positive labels : 25 out of total 142 datapoints


**Generate csv file for First Generation**

In [33]:
first_generation_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('data/csv/first_generation_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in first_generation_df.iterrows():
        text = val['Essay : Why am I here'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['First Gen - FINAL']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            

In [34]:
first_generation_sentence_df.shape

(128, 3)

In [35]:
pos_labels = len([n for n in first_generation_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(first_generation_sentence_df['label']), (pos_labels/len(first_generation_sentence_df['label']))*100))


Positive labels present in the dataset : 19  out of 128 or 14.84375%


In [36]:
train, test = train_test_split(first_generation_sentence_df, test_size=0.2, random_state=18, stratify=first_generation_sentence_df['label'])

In [37]:
train.to_csv("data/csv/first_generation_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [38]:
test.to_csv("data/csv/first_generation_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [39]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 14.71% positive class
positive labels : 15 out of total 102 datapoints


In [40]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 15.38% positive class
positive labels : 4 out of total 26 datapoints
